In [0]:
%run ./processor

In [0]:
%run ./schema_definition

In [0]:
# from schema_definition import schema, tracking_table_schema
import json
from botocore.exceptions import NoCredentialsError
from pyspark.sql import SparkSession
from datetime import datetime, timezone
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
import boto3

# Initialize Spark session
spark = SparkSession.builder.appName("S3_JSON_Loader").getOrCreate()

# Step 1 - Establish AWS connections
s3 = establish_aws_connection()

# Step 2 - Load the config and store the values into different variables
config = load_config_txt("config.txt")
bucket = config["bucket"]
prefix = config["prefix"]
files_tracking = config["files_tracking"]
last_ingested_times = config["last_ingested_times"]

# Step 3 - create tracking tables for files which have been ingested and
# the timestamp of the last ingested files (if not exists)
create_table(files_tracking, files_tracking_schema)
create_table(last_ingested_times, last_ingested_times_schema)

# Step 4: Read the last ingested timestamp
last_ingested_df = spark.read.table(last_ingested_times)
# Extract the only timestamp value
last_ingested_times = last_ingested_df.collect()[0]["last_ingested_times"]
# Convert to UTC 
if last_ingested_times.tzinfo is None:
    last_ingested_times = last_ingested_times.replace(tzinfo=timezone.utc)


# Step 5: List all files in S3
get_s3_data(s3, bucket, prefix, files_tracking, main_schema,last_ingested_times, files_tracking_schema)



